In [25]:
import os
import numpy as np
import mne
import sklearn
import matplotlib.pyplot as plt

from AudioOnsetUtils import *

RAW_EOG_CHANNELS = [u'EXG1', u'EXG2', u'EXG3', u'EXG4']
MASTOID_CHANNELS = [u'EXG5', u'EXG6']

In [26]:
## This cell gets filtered data so we can open ICA for Participant PNUM

# set the participant number
PNUM = "14"

data_raw_file = os.path.join('raw_data', 'P' + PNUM + '-raw.fif')
raw = mne.io.read_raw_fif(data_raw_file)

# remove mastoid channels if present 
if MASTOID_CHANNELS[0] in raw.ch_names:
    mne.io.set_eeg_reference(raw.load_data(), MASTOID_CHANNELS, copy=False) # inplace
    raw.drop_channels(MASTOID_CHANNELS)
    
# Drop bad channels - in place on raw
for bad_channel in raw.info['bads']:
    raw.drop_channels(bad_channel)
    print("dropped: " + bad_channel)
    
eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False, exclude=[])

# bandpass filter - keeping a frequency range between 0.5 (high pass filter) and 30 Hz (low pass filter)
filtered_data = raw.load_data().filter(0.5, 30, picks=eeg_picks)

Opening raw data file raw_data/P14-raw.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 66)  idle
    Range : 0 ... 2465433 =      0.000 ...  4815.299 secs
Ready.
Reading 0 ... 2465433  =      0.000 ...  4815.299 secs...
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Removing existing average EEG reference projection.
dropped: T7
dropped: F7
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 3381 samples (6.604 sec)



In [27]:
## This cell opens the ICA file for Participant NUM

ica = mne.preprocessing.read_ica('ica_data/P' + PNUM + '-ica.fif')
# apply the transformation
postica_data = ica.apply(filtered_data, exclude=ica.exclude)

Reading ica_data/P14-ica.fif ...
Isotrak not found
Now restoring ICA solution ...
Ready.
Applying ICA to Raw instance
    Transforming to ICA space (62 components)
    Zeroing out 6 ICA components
    Projecting back using 62 PCA components


In [28]:
events = mne.find_events(postica_data)
NUM_EVENTS = len(events) # num of first events we care about per participant

# creates tuples of (event_time, event_id)
event_times_ids = [(events[:][i][0], events[:][i][2]) for i in range(NUM_EVENTS)]
print(event_times_ids[:2])

540 events found
Event IDs: [  11   12   13   14   21   22   23   24   31   32   33   34   41   42
   43   44  111  112  113  114  121  122  123  124  131  132  133  134
  141  142  143  144  211  212  213  214  221  222  223  224  231  232
  233  234  241  242  243  244 1000 1111 2000 2001]
[(512, 121), (520, 1000)]


In [29]:
## Original code from https://github.com/sstober/openmiir-rl-2016/blob/master/openmiir/events.py#L21 and adapted 
KEYSTROKE_BASE_ID = 2000
TIME_INDEX = 0
ID_INDEX = 1

# returns two things, a True if Not Noise boolean and a tuple with three elements:
# 0- this is lyrical (0-2) <- YLABEL
# 1- start time (onsettime + cue duration if condition 1/2, else just onsettime)
# 2- end time (starttime + duration of song without cue)
def get_event_data(event_times_ids, events_index, music_version):
    event_id = event_times_ids[events_index[0]][ID_INDEX]

    if event_id < 1000:
        """
        Event Ids < 1000 are trial labels
        with the last digit indicating the condition
                1 : 'perception',
                2 : 'cued imag',
                3 : 'imag fix cross',
                4 : 'imagination',
        and the remaining digits referring to the stimulus id.
        """
        stimulus_id, condition = decode_event_id(event_id)
        
        stimulus_info = get_vers_stim_dict(music_version)[stimulus_id]
        cue_length = stimulus_info['cue_length']
        song_length = stimulus_info['song_length']
        
        events_index[0] += 1
        
        #print(get_id_to_song_name(stimulus_id))
        
        # get time of audio onset for this stimulus
        while event_times_ids[events_index[0]][ID_INDEX] != 1000:
            print("Expected an audio onset event but got {}".format(event_times_ids[events_index[0]]))
            events_index[0] += 1
            
        audio_onset_time = event_times_ids[events_index[0]][TIME_INDEX]
        start_time = (audio_onset_time + cue_length) if condition in [1,2] else audio_onset_time
        end_time = start_time + song_length
        
        # move the pointer by 1 to prep next call to get_event_data
        events_index[0] += 1
        
        return True, (stimulus_info["audio_type"], start_time, end_time)
    else:
        # move the pointer by 1 to prep next call to get_event_data
        events_index[0] += 1
        
        return {
            1111: (False, ("noise")),
            KEYSTROKE_BASE_ID: (False, 'imagination failed'),
            KEYSTROKE_BASE_ID+1: (False, 'imagination okay')
        }[event_id]
    
    
# convert event id to stimulus (song num) and condition (1-4)
def decode_event_id(event_id):
    if event_id < 1000:
        stimulus_id = event_id // 10
        condition = event_id % 10
        return stimulus_id, condition
    else:
        return event_id

In [30]:
events_index = [0]
music_version = get_participant_vers("01")
is_stim_event, event_data = get_event_data(event_times_ids, events_index, music_version)

In [156]:
ch_names = postica_data.ch_names[:-5]

(<AudioType.NONLYRICAL_LYRICAL: 1>, 521.869, 529.6427)

In [161]:
for i in range(len(ch_names)):
    channel_index = 0
    sampling_freq = raw.info['sfreq']
    start_stop_seconds = np.array([event_data[-2], event_data[-1]])
    start_sample, stop_sample = (start_stop_seconds * sampling_freq).astype(int)
    raw_selection = postica_data[channel_index, start_sample:(stop_sample+9)]
    #print(raw_selection)

61


In [48]:
def get_event_data_list(p_vers):
    events_data_list = []
    events_index = [0]
    music_version = get_participant_vers(p_vers)

    while len(events_data_list) < 50:
        is_audio, event_data = get_event_data(event_times_ids, events_index, music_version)
        if is_audio:
            # ignore AudioType.NONLYRICAL_LYRICAL (songs that have lyrics but played w/o lyrics)
            if event_data[0] == AudioType.LYRICAL or event_data[0] == AudioType.NON_LYRICAL:
                events_data_list.append(event_data)
    
    return events_data_list

In [49]:
get_event_data_list("01")

Expected an audio onset event but got (21617, 1111)
Expected an audio onset event but got (50133, 1111)
Expected an audio onset event but got (77942, 1111)
Expected an audio onset event but got (105307, 1111)
Expected an audio onset event but got (138780, 1111)
Expected an audio onset event but got (172603, 1111)
Expected an audio onset event but got (205513, 1111)
Expected an audio onset event but got (230208, 1111)
Expected an audio onset event but got (262803, 1111)
Expected an audio onset event but got (288607, 1111)
Expected an audio onset event but got (326592, 1111)
Expected an audio onset event but got (352259, 1111)
Expected an audio onset event but got (394194, 1111)
Expected an audio onset event but got (426737, 1111)
Expected an audio onset event but got (460552, 1111)
Expected an audio onset event but got (485290, 1111)
Expected an audio onset event but got (511043, 1111)
Expected an audio onset event but got (538356, 1111)
Expected an audio onset event but got (566933, 11

[(<AudioType.LYRICAL: 0>, 25506.317, 25515.9797),
 (<AudioType.LYRICAL: 0>, 34498.317, 34507.9797),
 (<AudioType.LYRICAL: 0>, 50270, 50279.6627),
 (<AudioType.NON_LYRICAL: 2>, 54023.307, 54032.5423),
 (<AudioType.NON_LYRICAL: 2>, 62761.307, 62770.5423),
 (<AudioType.NON_LYRICAL: 2>, 78119, 78128.2353),
 (<AudioType.LYRICAL: 0>, 176551.632, 176564.9481),
 (<AudioType.LYRICAL: 0>, 186826.632, 186839.9481),
 (<AudioType.LYRICAL: 0>, 205695, 205708.3161),
 (<AudioType.LYRICAL: 0>, 209453.831, 209461.5011),
 (<AudioType.LYRICAL: 0>, 217080.831, 217088.5011),
 (<AudioType.LYRICAL: 0>, 230390, 230397.6701),
 (<AudioType.LYRICAL: 0>, 234156.988, 234168.6112),
 (<AudioType.LYRICAL: 0>, 244472.988, 244484.6112),
 (<AudioType.LYRICAL: 0>, 262969, 262980.6232),
 (<AudioType.NON_LYRICAL: 2>, 266740.995, 266749.3215),
 (<AudioType.NON_LYRICAL: 2>, 274676.995, 274685.3215),
 (<AudioType.NON_LYRICAL: 2>, 288813, 288821.3265),
 (<AudioType.NON_LYRICAL: 2>, 292559.182, 292575.18409999995),
 (<AudioType.